In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install tf2onnx


In [ ]:
import torch
import torch.nn as nn
import time
import torch.onnx
import tensorflow as tf
import onnxruntime as ort
import numpy as np
import tf2onnx
print(torch.cuda.is_available())

In [ ]:

# Define a simple feedforward network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(2, 2)  # Input to layer 1
        self.fc2 = nn.Linear(2, 3)  # Layer 1 to layer 2
        self.fc3 = nn.Linear(3, 4)  # Layer 2 to layer 3 (output)

    def forward(self, x):
        # Forward pass through the network
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)  # Output layer has no activation
        return x

# Instantiate the model
model_pt = SimpleNet()

# Example input for inference
example_input_pt = torch.tensor([[6.0, 9.0]])

# Perform inference
output = model_pt(example_input_pt)

# Print the inference output
print("PyTorch Inference Output:", output.detach().numpy())

In [ ]:
# Define the model using TensorFlow Sequential API
model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation='sigmoid', input_shape=(2,)),  # Input to layer 1
    tf.keras.layers.Dense(3, activation='sigmoid'),                     # Layer 1 to layer 2
    tf.keras.layers.Dense(4)                                            # Layer 2 to output (no activation)
])

# Example input for inference
example_input_tf = tf.constant([[6.0, 9.0]], dtype=tf.float32)

# Perform inference
output = model_tf(example_input)

# Print the inference output
print("TensorFlow Inference Output:", output.numpy())

In [ ]:
# Specify the path for the ONNX model file
onnx_model_path_pt = "simple_model_pt.onnx"
onnx_model_path_tf = "simple_model_tf.onnx"
# Convert the PyTorch model to ONNX
torch.onnx.export(
    model_pt,                          # model being exported
    example_input_pt,                  # model input (or a tuple for multiple inputs)
    onnx_model_path_pt,                # where to save the model (can be a file or file-like object)
    input_names=["input_pt"],          # the model's input names
    output_names=["output_pt"],        # the model's output names
)

print(f"pytorch Model successfully converted to ONNX: {onnx_model_path_pt}")
#########################################################################################
spec = (tf.TensorSpec((None, 2), tf.float32, name="input_tf"),)  # Define input shape and name

# Convert and save the model
tf2onnx.convert.from_keras(model_tf, 
                            input_signature=spec, 
                            output_path=onnx_model_path_tf)



print(f"tensorflow Model successfully converted to ONNX: {onnx_model_path_tf}")



In [ ]:
# Load the ONNX model
onnx_model_path = "simple_model_pt.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Prepare sample input data (same shape as the PyTorch model)
onnx_input = np.array([[1.0, 2.0]], dtype=np.float32)

# Run inference on the ONNX model
onnx_output = ort_session.run(None, {"input_pt": onnx_input})

# Print the ONNX inference result
print("ONNX Inference Output pytorch model:", onnx_output)


In [ ]:
# Load the ONNX model
onnx_model_path = "simple_model_tf.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Prepare sample input data (same shape as the PyTorch model)
onnx_input = np.array([[1.0, 2.0]], dtype=np.float32)

# Run inference on the ONNX model
onnx_output = ort_session.run(None, {"input_tf": onnx_input})

# Print the ONNX inference result
print("ONNX Inference Output using tensorflow model:", onnx_output)

In [ ]:
time_sum = 0
n_iters = 1000

for i in range(n_iters):
  example_input = torch.randn(1, 2)
  t0 = time.time()
  output = model(example_input)
  time_sum+=(time.time() - t0)

print(f"Total time: {time_sum/n_iters}")

In [ ]:
time_sum = 0
n_iters = 1000
onnx_model_path = "simple_model.onnx"
ort_session = ort.InferenceSession(onnx_model_path)


for i in range(n_iters):
  onnx_input = np.random.rand(1, 2).astype(np.float32)
  t0 = time.time()
  onnx_output = ort_session.run(None, {"input": onnx_input})
  time_sum+=(time.time() - t0)

print(f"Total time: {time_sum/n_iters}")